# CVS Health Community Access Analysis - Part 5: K-Means Clustering Analysis

This notebook uses K-means clustering to identify distinct groups of counties based on their characteristics. This helps us create targeted expansion strategies for similar county types.


## Understanding K-Means Clustering

K-means clustering is an unsupervised machine learning algorithm that groups similar data points together. Here's how it works:

1. **Choose K clusters**: We select the number of groups we want to identify
2. **Initialize centroids**: Random starting points for each cluster
3. **Assign points**: Each county is assigned to the nearest cluster centroid
4. **Update centroids**: The center of each cluster is recalculated
5. **Repeat**: Steps 3-4 are repeated until clusters stabilize

**For our analysis**, we'll cluster counties based on:
- Health burden (how sick the population is)
- Social vulnerability (socioeconomic challenges)
- Clinic access (current CVS presence)
- Population size (market size)

This will help us identify:
- **High-priority expansion targets**: High need, high population, low access
- **Maintenance markets**: Already well-served areas
- **Rural underserved**: High need but small population
- **Urban opportunities**: Large population with moderate need


In [ ]:
# import libraries for clustering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# install scikit-learn if needed
try:
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import silhouette_score
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import silhouette_score

print("libraries imported successfully")


## Prepare Data for Clustering

we need to ensure all required columns exist and prepare the data for clustering. we'll use health burden, SVI, clinic count, and population as features.


In [ ]:
# ensure required columns exist before clustering
if 'health_burden_score' not in df.columns:
    health_vars = ['stroke', 'physical_inactivity', 'self_care_disability', 'social_isolation']
    available_vars = [var for var in health_vars if var in df.columns]
    if available_vars:
        df['health_burden_score'] = df[available_vars].mean(axis=1)
        print(f"created health_burden_score from: {available_vars}")
    else:
        raise ValueError("cannot create health_burden_score - required health variables missing")

if 'population' not in df.columns:
    raise ValueError("'population' column not found. please run notebook 06c to add population data first.")

# prepare features for clustering
# we'll use normalized versions of key metrics
clustering_features = ['health_burden_score', 'svi_overall', 'clinic_count', 'population']

# handle missing values and infinite values
df_cluster = df[clustering_features + ['county_full', 'state_full', 'fips']].copy()
df_cluster = df_cluster.dropna()

# replace infinite values with NaN and drop
df_cluster = df_cluster.replace([np.inf, -np.inf], np.nan).dropna()

# log transform population to reduce skewness
# population is highly skewed (few very large counties), so we use log to normalize
df_cluster['log_population'] = np.log1p(df_cluster['population'])

# prepare features for clustering (use log population instead of raw)
X = df_cluster[['health_burden_score', 'svi_overall', 'clinic_count', 'log_population']].values

# standardize features (important for K-means)
# this ensures all features are on the same scale so no single feature dominates
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nprepared {len(X_scaled)} counties for clustering")
print(f"features: Health Burden, SVI Overall, Clinic Count, Log Population")


## Determine Optimal Number of Clusters

we need to decide how many clusters to use. we'll test different values and use the elbow method and silhouette score to find the best number.


In [ ]:
# test different numbers of clusters
# we'll test K values from 2 to 10
inertias = []  # within-cluster sum of squares (lower is better)
silhouette_scores = []  # measure of cluster quality (higher is better, range -1 to 1)
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# plot elbow curve and silhouette scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# elbow method: look for the "elbow" where inertia stops decreasing rapidly
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=11)
axes[0].set_ylabel('Inertia (Within-cluster sum of squares)', fontsize=11)
axes[0].set_title('Elbow Method for Optimal K', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)

# silhouette score: higher is better
axes[1].plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=11)
axes[1].set_ylabel('Silhouette Score', fontsize=11)
axes[1].set_title('Silhouette Score for Optimal K', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# find optimal K (highest silhouette score)
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\noptimal number of clusters: {optimal_k} (Silhouette Score: {max(silhouette_scores):.3f})")
print("\nsilhouette scores by K:")
for k, score in zip(K_range, silhouette_scores):
    print(f"  K={k}: {score:.3f}")


### what these plots show:

- **elbow method (left plot)**: shows how much the within-cluster variance decreases as we add more clusters. the "elbow" is where adding more clusters doesn't help much. lower inertia is better, but we want a balance.

- **silhouette score (right plot)**: measures how well-separated clusters are. values range from -1 to 1, where 1 means perfect separation. higher is better. we choose the K with the highest silhouette score.

- the optimal K balances having enough clusters to capture patterns but not so many that clusters become meaningless.


## Perform K-Means Clustering

now we'll perform the actual clustering using the optimal number of clusters we determined.


In [ ]:
# use optimal K (or 5 if we want more granular clusters)
# we use at least 4 clusters to get meaningful segments
n_clusters = optimal_k if optimal_k >= 4 else 5

# perform K-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df_cluster['cluster'] = kmeans.fit_predict(X_scaled)

# calculate cluster characteristics
cluster_summary = df_cluster.groupby('cluster').agg({
    'health_burden_score': 'mean',
    'svi_overall': 'mean',
    'clinic_count': 'mean',
    'population': 'mean',
    'county_full': 'count'
}).round(3)
cluster_summary.columns = ['Avg Health Burden', 'Avg SVI', 'Avg Clinic Count', 'Avg Population', 'County Count']

print(f"K-Means Clustering Results ({n_clusters} clusters):")
print("=" * 80)
print(cluster_summary.to_string())
print("\nsilhouette score:", silhouette_score(X_scaled, df_cluster['cluster']))


### what this clustering shows:

- each cluster represents a distinct type of county with similar characteristics
- clusters with high health burden and low clinic count are priority expansion targets
- clusters with high population and low clinic count represent urban opportunities
- clusters with low health burden and high clinic count are well-served markets
- we can develop targeted strategies for each cluster type


## Visualize Clusters

we'll create scatter plots to visualize how counties are grouped into clusters. this helps us understand the relationships between features and see cluster patterns.


In [ ]:
# create visualizations of clusters
# we'll show clusters from different angles to understand the patterns
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# plot 1: health burden vs SVI (colored by cluster)
scatter1 = axes[0, 0].scatter(df_cluster['health_burden_score'], df_cluster['svi_overall'], 
                              c=df_cluster['cluster'], cmap='tab10', alpha=0.6, s=50)
axes[0, 0].set_xlabel('Health Burden Score', fontsize=11)
axes[0, 0].set_ylabel('Social Vulnerability Index', fontsize=11)
axes[0, 0].set_title('Cluster Visualization: Health Burden vs SVI', fontsize=12, fontweight='bold')
axes[0, 0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0, 0], label='Cluster')

# plot 2: population vs clinic count (colored by cluster)
scatter2 = axes[0, 1].scatter(df_cluster['population']/1e6, df_cluster['clinic_count'], 
                              c=df_cluster['cluster'], cmap='tab10', alpha=0.6, s=50)
axes[0, 1].set_xlabel('Population (Millions)', fontsize=11)
axes[0, 1].set_ylabel('Clinic Count', fontsize=11)
axes[0, 1].set_title('Cluster Visualization: Population vs Clinic Count', fontsize=12, fontweight='bold')
axes[0, 1].grid(alpha=0.3)
plt.colorbar(scatter2, ax=axes[0, 1], label='Cluster')

# plot 3: health burden vs clinic count (colored by cluster)
scatter3 = axes[1, 0].scatter(df_cluster['health_burden_score'], df_cluster['clinic_count'], 
                              c=df_cluster['cluster'], cmap='tab10', alpha=0.6, s=50)
axes[1, 0].set_xlabel('Health Burden Score', fontsize=11)
axes[1, 0].set_ylabel('Clinic Count', fontsize=11)
axes[1, 0].set_title('Cluster Visualization: Health Burden vs Clinic Count', fontsize=12, fontweight='bold')
axes[1, 0].grid(alpha=0.3)
plt.colorbar(scatter3, ax=axes[1, 0], label='Cluster')

# plot 4: cluster sizes (bar chart)
cluster_sizes = df_cluster['cluster'].value_counts().sort_index()
axes[1, 1].bar(cluster_sizes.index, cluster_sizes.values, color=plt.cm.tab10(range(len(cluster_sizes))))
axes[1, 1].set_xlabel('Cluster ID', fontsize=11)
axes[1, 1].set_ylabel('Number of Counties', fontsize=11)
axes[1, 1].set_title('Cluster Sizes', fontsize=12, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(cluster_sizes.values):
    axes[1, 1].text(cluster_sizes.index[i], v + 10, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()


### what these visualizations show:

- **top left**: shows how health burden and SVI relate, with clusters colored differently. clusters in the top-right (high burden, high SVI) are most vulnerable.

- **top right**: shows population vs clinic count. clusters in the bottom-right (high population, low clinics) are urban expansion opportunities.

- **bottom left**: directly shows the need-access relationship. clusters in the top-left (high burden, low clinics) are priority targets.

- **bottom right**: shows how many counties are in each cluster. balanced clusters are better than one very large cluster.

- colors help identify which counties belong to the same cluster across different views.


## Cluster Interpretation and Business Recommendations

now we'll interpret each cluster and provide specific expansion recommendations based on cluster characteristics.


In [ ]:
# identify cluster characteristics and assign labels
cluster_labels = {}

for cluster_id in sorted(df_cluster['cluster'].unique()):
    cluster_data = df_cluster[df_cluster['cluster'] == cluster_id]
    
    avg_health = cluster_data['health_burden_score'].mean()
    avg_svi = cluster_data['svi_overall'].mean()
    avg_clinics = cluster_data['clinic_count'].mean()
    avg_pop = cluster_data['population'].mean()
    zero_clinic_pct = (cluster_data['clinic_count'] == 0).mean() * 100
    
    # determine cluster type based on characteristics
    if avg_health > df['health_burden_score'].quantile(0.75) and avg_clinics < 1:
        if avg_pop > df['population'].quantile(0.75):
            label = "HIGH PRIORITY: Urban High-Need, Low-Access"
            priority = "CRITICAL"
        else:
            label = "Rural High-Need, Low-Access"
            priority = "HIGH"
    elif avg_health > df['health_burden_score'].quantile(0.75) and avg_clinics >= 1:
        label = "Well-Served High-Need"
        priority = "LOW"
    elif avg_pop > df['population'].quantile(0.75) and avg_clinics < 2:
        label = "Large Population, Moderate Access"
        priority = "MEDIUM"
    elif avg_svi > df['svi_overall'].quantile(0.75) and avg_clinics < 1:
        label = "High Vulnerability, Low Access"
        priority = "HIGH"
    else:
        label = "Other Markets"
        priority = "LOW"
    
    cluster_labels[cluster_id] = {
        'label': label,
        'priority': priority,
        'counties': len(cluster_data),
        'avg_health': avg_health,
        'avg_svi': avg_svi,
        'avg_clinics': avg_clinics,
        'avg_pop': avg_pop,
        'zero_clinic_pct': zero_clinic_pct
    }

# print cluster interpretations
print("\n" + "=" * 100)
print("CLUSTER INTERPRETATIONS AND EXPANSION RECOMMENDATIONS")
print("=" * 100)

for cluster_id in sorted(cluster_labels.keys()):
    info = cluster_labels[cluster_id]
    print(f"\n{'='*100}")
    print(f"CLUSTER {cluster_id}: {info['label']}")
    print(f"Priority Level: {info['priority']}")
    print(f"{'='*100}")
    print(f"  • Number of Counties: {info['counties']}")
    print(f"  • Average Health Burden: {info['avg_health']:.2f}")
    print(f"  • Average SVI: {info['avg_svi']:.3f}")
    print(f"  • Average Clinic Count: {info['avg_clinics']:.2f}")
    print(f"  • Average Population: {info['avg_pop']:,.0f}")
    print(f"  • Counties with Zero Clinics: {info['zero_clinic_pct']:.1f}%")
    
    # show top counties in this cluster
    cluster_counties = df_cluster[df_cluster['cluster'] == cluster_id].copy()
    cluster_counties['priority_metric'] = cluster_counties['health_burden_score'] * cluster_counties['population']
    cluster_counties = cluster_counties.nlargest(5, 'priority_metric')
    if len(cluster_counties) > 0:
        print(f"\n  Top 5 Counties in This Cluster (by Health Need × Population):")
        for idx, row in cluster_counties.iterrows():
            print(f"    - {row['county_full']}, {row['state_full']} "
                  f"(Pop: {row['population']:,.0f}, Clinics: {row['clinic_count']:.0f}, "
                  f"Health Burden: {row['health_burden_score']:.2f})")
